In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from tqdm import tqdm
tqdm.pandas()
%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
from datasets import Dataset
import pandas as pd
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion,
  extract_first
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin
/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/deepeval/__init__.py:41: UserWarning: You are using deepeval version 0.20.90, however version 0.21.15 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [4]:
from typing import Callable
from tqdm import tqdm

In [5]:
# subdependent 
from lm_act_eval.evaluation_harness.evaluators.sft.base import DataFrameEvaluator
from lm_act_eval.evaluation_harness.helper_functions import multion

In [6]:
from lm_act_eval._init_conf import init_conf
hydra_cfg = init_conf('trajectory_eval')
hydra_cfg

/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/_init_conf.py:9: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../config")


{'sft': {'trajectory': {'data': {'path': 'lm_act_eval/.cache/five-star-trajectories/csv/data+gptv-eligible.csv', 'columns': {'y': 'ground_truth', "y_'": 'GPTV_generations'}, 'extract_fs': {'QUERY': {'QUERY': None}, 'screenshot': {'screenshot': None}, 'GOAL': {'chat_completion_messages': 'parse_completion.parse_content'}}, 'logging': {'project': 'opentable', 'result': 'lm_act_eval-run', 'hugging': {'space': 'multion-agi'}}}, 'comparator': {'gpt-v': {'model': 'gpt-4-vision-preview', 'max_token': 300, 'img_fidelity': 'high'}}, 'metrics': {'gpt-v': {'inputs': ['GOAL', 'QUERY', 'screenshots'], 'args': None}, 'llm_relevancy': ['explanation']}}}}

In [7]:
df = pd.read_csv(hydra_cfg.sft.trajectory.data.path)
print(
  df.chat_completion_messages.iloc[0][:100],
  df.columns
)

[{'role': 'system', 'content': '**Goal**: Let\'s play a game - You are an expert agent named MULTI·O Index(['Unnamed: 0', 'session_id', 'idx_in_session', '_id', 'event_id',
       'parent_id', 'event_type', 'event_name', 'config', 'children_ids',
       'inputs', 'outputs', 'error', 'start_time', 'end_time', 'metadata',
       'feedback', 'metrics', 'user_properties', 'source', 'duration',
       'project_id', 'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata',
       'chat_completion_messages_content', 'GPTV_generations'],
      dtype='object')


In [8]:
from lm_act_eval.evaluation_harness.openai.vision.gptv import GPTV
from lm_act_eval.evaluation_harness.openai.vision.config import gptv_config

from lm_act_eval.evaluation_harness.openai.vision.evaluator import GPTVScorer

In [38]:
from lm_act_eval.evaluation_harness.evaluators.sft.trajectory import TableTrajectoryEvaluator

df_evaluator = TableTrajectoryEvaluator(hydra_cfg.sft.trajectory.data)

/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/utils.py:53: UserWarning: DO NOTHING doesn't exist in the registry.
  warnings.warn(f"{name} doesn't exist in the registry.")
/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/utils.py:53: UserWarning: DO NOTHING doesn't exist in the registry.
  warnings.warn(f"{name} doesn't exist in the registry.")
/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:136: UserWarning: Failed to parse json, returning content field
  warnings.warn(f"Failed to parse json, returning content field")
/home/opadmin/llama2d_opentable/lm_act_eval/lm_act_eval/evaluation_harness/helper_functions/multion.py:154: UserWarning: Unexpected format, returning suspicious content of type <class 'str'>, using as is
  warnings.warn(f"Unexpected format, returning suspicious content of type {type(chat_completion_msgs)}, using as is")


In [23]:
df_evaluator.df.head(2)

,QUERY,screenshot,GOAL
169,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...
170,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...


In [56]:
gptv_scorer = GPTVScorer(hydra_cfg.sft.trajectory.metrics['gpt-v'])
hydra_cfg.sft.trajectory.metrics['gpt-v']

{'inputs': ['GOAL', 'QUERY', 'screenshots'], 'args': None}

In [43]:
# 
df_evaluator.df.iloc[:2]

,QUERY,screenshot,GOAL
169,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...
170,What was the actual enrollment count of the cl...,https://multion-client-screenshots.s3.us-east-...,**Goal**: Let\'s play a game - You are an expe...


In [57]:
gptv_scorer(df_evaluator.df.iloc[:2])

Determining entry eligibility:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating with GPT-V: 100%|██████████| 2/2 [00:19<00:00,  9.98s/it]

completions:SCORE: 20
EXPLANATION:
The screenshot provided shows that the user is on the Google search results page with a search query that appears to be related to the objective of finding the enrollment count of a clinical trial regarding H. pylori in acne vulgaris patients from January to May 2018. The search results include links to ClinicalTrials.gov, which is the correct source for finding detailed information on NIH clinical trials. Since ClinicalTrials.gov is the NIH repository for clinical trial data, the user is on the right path.

However, the user has not yet clicked on any of the links that lead to the specific clinicaltrial record which would contain the actual enrollment count. The score reflects that the user has correctly identified where to find the information but has not yet accessed the exact data required to complete the task.

To proceed, the user would need to perform a few more steps:
1. Click on the relevant link (most likely the first one, as it directly ref

Evaluating with GPT-V: 100%|██████████| 2/2 [00:50<00:00, 25.03s/it]

completions:SCORE: 95
EXPLANATION: 
The screenshot provided is of the ClinicalTrials.gov listing for the study in question, "Prevalence of H.Pylori in Patients With Acne Vulgaris." This indicates that the user has successfully navigated to a page that is highly likely to contain the desired information about the enrollment count of the clinical trial. However, the specific piece of information regarding the actual enrollment count is not visible in the screenshot provided. To reach a score of 100, the user would need to locate and view that specific detail. Given the current page content with its study description and identifiers, the user is very close to completing the task and would typically need to either scroll through the current page or click on a link that leads to more detailed information about the study, such as the enrollment count. The "No Results Posted" section may also suggest that a further action, like clicking to view more details or using a different view (such as 

,Score,Explanation
169,20,The screenshot provided shows that the user is...
170,95,The screenshot provided is of the ClinicalTria...


In [ ]:
# Using as agent
from lm_act_eval.evaluation_harness.helper_functions.multion import extract_explanation

In [ ]:
# evaluator prompt
from lm_act_eval.evaluation_harness.openai.vision.prompts import DEFAULT_EVAL_PROMPT
eval_query = eval_input_df.apply(lambda r: DEFAULT_EVAL_PROMPT.format(**r), axis=1)
# extract_explanation = lambda x: re.search(r'EXPLANATION:\n(.+)', x, re.DOTALL).group(1) if re.search(r'EXPLANATION:\n', x) else None


NameError: name 'eval_input_df' is not defined

In [ ]:
input_df = pd.DataFrame(index=eligible_traj_df.index)
input_df['text'] = eval_query
input_df['images'] = screenshots

In [ ]:
import re
evals = input_df.iloc[:5].progress_apply(lambda r: gptv_pipeline.generate_completion(**r), axis=1)

NameError: name 'input_df' is not defined

In [ ]:
evals

Unnamed: 0
169    SCORE: 90\nEXPLANATION:\nThe user is very clos...
170    SCORE: 95\nEXPLANATION:\nThe user is very clos...
171    SCORE: 90\nEXPLANATION:\nThe user has successf...
172    SCORE: 100\nEXPLANATION:\nThe user has success...
408    SCORE: 95\nEXPLANATION:\nThe user is very clos...
dtype: object

In [ ]:
result_df = evals.str.split('\n', n=1, expand=True).rename_axis(index=None)
result_df.columns = ['Score', 'Explanation']
result_df.head(2)

In [ ]:
result_df['Score'] = result_df['Score'].apply(lambda s: extract_first(s, term='SCORE'))
extract_explanation = lambda x: re.search(r'EXPLANATION:\n(.+)', x, re.DOTALL).group(1) if re.search(r'EXPLANATION:\n', x) else None
result_df['Explanation'] = result_df['Explanation'].apply(lambda s: extract_explanation(s))

NameError: name 'result_df' is not defined

In [ ]:
print(result_df.Explanation.iloc[0])

The user is very close to completing the task. They have successfully navigated to the Google search query page that might contain links to the NIH website where the information about the clinical trial can be found. However, the user has not yet clicked on the relevant links that would take them directly to the NIH website to check the actual enrollment count of the clinical trial on H. pylori in acne vulgaris patients for the specified period.

The next step would be to follow the links provided in the search results to the clinicaltrials.gov entries that appear to contain the desired information. Since the exact URL or the identity of the clickable link in the search results that leads directly to the information on the NIH website is not visible in the provided context, it can't be considered as completely done. A little more navigation within the search results is needed to reach the exact page that holds the enrollment count.

STATUS: CONTINUE

COMMANDS:
CLICK 1
EXPLANATION: I am

In [ ]:
eligible_traj_df.loc[:, 'GPTV_evaluations'] = completion_responses

## Save/Load Data

In [ ]:
# traj_df_new = traj_df.merge(eligible_traj_df, how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv_eval.csv')

In [ ]:
table_name = 'data+gptv_eval-eligible'
artifact_name = 'gptv_opentable-trajectory_eval'
eligible_traj_df.to_csv(dataset_path/f'csv/{table_name}.csv')

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/f'csv/{table_name}.csv', index_col=0)
eligible_traj_df.index.name = None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,GPTV response,chat_completion_messages_content,GPTV_evaluation
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,It appears that you are looking for informatio...,**Goal**: Let's play a game - You are an exper...,SCORE: 60\nEXPLANATION:\nThe user appears to b...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,The image shows a webpage from ClinicalTrials....,**Goal**: Let's play a game - You are an exper...,SCORE: 95\nEXPLANATION:\nThe user is very clos...


Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories-gptv_eval", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(
  opentable_table_eligible, artifact_name)
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/f'csv/{table_name}.csv'))


In [ ]:
# Start a W&B run to log data
run = wandb.init(project="trajectory_eval", reinit=True)
# Log the table to visualize with a run...
run.log({"opentable_gptv": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: Currently logged in as: mmym (multion-agi). Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories-gptv_eval>